### Pools
| Pair | Uniswap Pool | Binance Spot 
| -----|--------------|----------------
| ETH/USDT|https://info.uniswap.org/#/pools/0x11b815efb8f581194ae79006d24e0d814b7697f6|https://www.binance.com/en/trade/ETH_USDT
| WBTC/USDT|https://info.uniswap.org/#/pools/0x9db9e0e53058c89e5b94e29621a205198648425b|https://www.binance.com/en/trade/BTC_USDT
| UNI/USDC |https://info.uniswap.org/#/pools/0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78|https://www.binance.com/en/trade/UNI_USDT



APIs: Binance: https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data

Web3: https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.get_logs



**Before starting create config.yaml of following format:**
```
keys:
    infura_api : "YOUR-API-KEY"
```

# Get Uniswap Data

#### Imports

In [2]:
import yaml
import traceback
import pandas as pd
from web3 import Web3
from eth_abi import abi

#### Prepare W3 Connection

In [2]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

def conETH(infura_api):
    # url link to the ethereum node
    url_eth_mainnet = "https://mainnet.infura.io/v3/"
    try:
        # connect to the ethereum node
        w3 = Web3(Web3.HTTPProvider(url_eth_mainnet + infura_api))
        return w3
    except:
        return None

if config["keys"]["infura_api"] is not None:
    w3 = conETH(config["keys"]["infura_api"])
    # make the query
    if w3 is not None:
      print("Connect Successful")
    else:
      print("Error: connection to the ethereum node failed")
else:
    print("Error: infura api key not found")

Connect Successful


#### Reusable Methods

In [3]:
SWAP_TOPIC = '0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67'

def getLogs(start, nr, pool_addr):
  try:
    return w3.eth.get_logs({'fromBlock': start, 'toBlock': start+nr, 'address': w3.to_checksum_address(pool_addr)})
  except:
    traceback.print_exc()
    return None

def decodeRow(row):
  decoded = abi.decode(['int256', 'int256', 'uint160', 'uint128', 'int24'], row['data'])
  row['amount0'] = decoded[0]
  row['amount1'] = decoded[1]
  row['sqrtPriceX96'] = decoded[2]
  row['txHash'] = row['transactionHash'].hex()
  # row['liquidity'] = decoded[3]
  # row['tick'] = decoded[4]
  return row

def decode_hex_data_from_64b_log (hexstr):
    chunk_size = 64
    # Split the hex string into chunks of 32 bytes
    hex_chunks = [hexstr[i:i + chunk_size] for i in range(2, len(hexstr), chunk_size)]
    # Convert each chunk to decimal
    decimal_chunks = [float(chunk) for chunk in hex_chunks]
    return decimal_chunks

def filter_and_decode(entries, pool_addr):
  df_logs = pd.DataFrame(entries)
  # Has the columns: Index(['address', 'blockHash', 'blockNumber', 'data', 'logIndex', 'removed', 'topics', 'transactionHash', 'transactionIndex']

  print("Entries before address and topic filter:",len(df_logs))
  # Retrieve lead topic and filter for address and swap event
  df_logs = df_logs.drop_duplicates(subset='blockNumber', keep='last')
  df_logs.loc[:, "topic"] = df_logs.topics.apply(lambda s: s[0].hex())
  df_logs1 = df_logs.loc[(df_logs["address"].str.casefold() == pool_addr) & (df_logs["topic"] == SWAP_TOPIC)]
  print("Entries after filters:",len(df_logs1))

  # Decode the log data
  df_logs1 = df_logs1.apply(decodeRow, axis=1)
  df_logs1 = df_logs1[['address', 'blockNumber', 'txHash','amount0','amount1','sqrtPriceX96']]
  #print(df_logs1)
  
  return df_logs1


def GetPrice(sqrtPriceX96, Decimal0, Decimal1):
    buyOneOfToken0 = ((sqrtPriceX96 / 2**96)**2) / (10**Decimal1 / 10**Decimal0)

    return buyOneOfToken0

    # buyOneOfToken1 = 1 / buyOneOfToken0
    # print("price of token0 in value of token1 : " + str(buyOneOfToken0))
    # print("price of token1 in value of token0 : " + str(buyOneOfToken1))
    # print("")
    # # Convert to wei
    # buyOneOfToken0Wei = str(int(buyOneOfToken0 * (10**Decimal1)))
    # buyOneOfToken1Wei = str(int(buyOneOfToken1 * (10**Decimal0)))
    # print("price of token0 in value of token1 in lowest decimal : " + buyOneOfToken0Wei)
    # print("price of token1 in value of token1 in lowest decimal : " + buyOneOfToken1Wei)
    # print("")'
def GetSwapPrice(amount0, amount1, Decimal0, Decimal1):
    buyOneOfToken0 = ((amount1 / 10**Decimal1) / (amount0 / 10**Decimal0)) * -1

    return buyOneOfToken0

def first(row):
    return row.iloc[0]
def last(row):
    return row.iloc[-1]

In [4]:
# Load Block Timestamps from CSV
timestamps = pd.read_csv('block_timestamps.csv')

def extractPrices(logs, addr, decimal0, decimal1):
  swap_logs = filter_and_decode(logs, addr)
  # Merge Timestamps with Swap Log
  swap_logs = pd.merge(swap_logs, timestamps, left_on='blockNumber', right_on='number', how='left')

  # Add datetime format timestamp
  swap_logs['datetime'] = pd.to_datetime(swap_logs['timestamp'], unit='s')
  swap_logs = swap_logs.set_index('datetime')

  # Calculate Price of Token0 in Token1
  swap_logs['price'] = swap_logs['sqrtPriceX96'].apply(lambda price: GetPrice(price,decimal0,decimal1))
  return swap_logs[['blockNumber', 'price']]

def fillAllBlocks(prices):
  result = pd.merge(timestamps, prices, left_on='number', right_on='blockNumber', how='left')
  maxblock = prices['blockNumber'].max()
  result = result[result['number'] <= maxblock]
  result['datetime'] = pd.to_datetime(result['timestamp'], unit='s')
  result = result.set_index('datetime')
  result['price'] = result['price'].fillna(method='ffill')
  return result[['number', 'price']]

def extractCandles(prices):
  # Resample txHash
  candles = prices.resample('1T').agg({'price': [('open','first'),('high','max'), ('low','min'), ('close','last')]})
  candles.columns = ['open','high','low','close']

  # Forward Fill Close Price
  candles['close'].ffill(inplace=True)

  # Cross Fill High and Low from Close Price
  candles['high'].fillna(candles['close'],inplace=True)
  candles['low'].fillna(candles['close'],inplace=True)

  # Starting from second row, fill the open price with the previous close price
  candles.iloc[1:,0] = candles['close'].iloc[0:-1]

  return candles
  # time_df

#### Process Data in Loop

In [5]:
SYMBOL = "WETH_USDT"
POOL_ADDR = '0x11b815efb8f581194ae79006d24e0d814b7697f6'
DECIMAL_0 = 18
DECIMAL_1 = 6
CHUNK_SIZE = 7000

# SYMBOL = "WBTC_USDT"
# POOL_ADDR = '0x9db9e0e53058c89e5b94e29621a205198648425b'
# DECIMAL_0 = 8
# DECIMAL_1 = 6
# CHUNK_SIZE = 50000

# SYMBOL = "UNI_USDC"
# POOL_ADDR = '0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78'
# DECIMAL_0 = 18
# DECIMAL_1 = 6
# CHUNK_SIZE = 50000

START_BLOCK = 13916166
END_BLOCK = 13917166
# END_BLOCK = 16308189


bl = START_BLOCK
candles = pd.DataFrame()
while bl <= END_BLOCK:
    chunk_size = min(CHUNK_SIZE,END_BLOCK-bl+1)
    print("Getting logs from block",bl,"to",bl+chunk_size-1)
    logs = getLogs(bl,chunk_size,POOL_ADDR)
    print("Received",len(logs), "tx logs")
    if (len(logs) > 0):
      prices = extractPrices(logs,POOL_ADDR,DECIMAL_0,DECIMAL_1)
      new_candles = extractCandles(prices)
      candles = pd.concat([candles,extractPrices(logs,POOL_ADDR,DECIMAL_0,DECIMAL_1)],axis=0)
      candles.to_pickle(f'./dex_{SYMBOL}_2022.pkl')
    bl += CHUNK_SIZE

Getting logs from block 13916166 to 13917166
Received 243 tx logs
Entries before address and topic filter: 243
Entries after filters: 192
Entries before address and topic filter: 243
Entries after filters: 192


In [6]:
# SYMBOL = "WETH_USDT"
# POOL_ADDR = '0x11b815efb8f581194ae79006d24e0d814b7697f6'
# DECIMAL_0 = 18
# DECIMAL_1 = 6
# CHUNK_SIZE = 7000

# SYMBOL = "WBTC_USDT"
# POOL_ADDR = '0x9db9e0e53058c89e5b94e29621a205198648425b'
# DECIMAL_0 = 8
# DECIMAL_1 = 6
# CHUNK_SIZE = 50000

# SYMBOL = "UNI_USDC"
# POOL_ADDR = '0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78'
# DECIMAL_0 = 18
# DECIMAL_1 = 6
# CHUNK_SIZE = 50000

SYMBOL = "SHIB_ETH"
POOL_ADDR = '0x5764a6f2212d502bc5970f9f129ffcd61e5d7563'
DECIMAL_0 = 18
DECIMAL_1 = 18
CHUNK_SIZE = 50000

START_BLOCK = 13916166
END_BLOCK = 16308189
# END_BLOCK = 16308189


bl = START_BLOCK
prices = pd.DataFrame()
while bl <= END_BLOCK:
    chunk_size = min(CHUNK_SIZE,END_BLOCK-bl+1)
    print("Getting logs from block",bl,"to",bl+chunk_size-1)
    logs = getLogs(bl,chunk_size,POOL_ADDR)
    print("Received",len(logs), "tx logs")
    if (len(logs) > 0):
      new_prices = extractPrices(logs,POOL_ADDR,DECIMAL_0,DECIMAL_1)
      prices = pd.concat([prices,new_prices],axis=0)
      # prices.to_pickle(f'./dex_{SYMBOL}_2022_allblocks.pkl')
    bl += CHUNK_SIZE

prices = fillAllBlocks(prices)
prices.to_pickle(f'./dex_{SYMBOL}_2022_allblocks.pkl')

Getting logs from block 13916166 to 13966165
Received 199 tx logs
Entries before address and topic filter: 199
Entries after filters: 133
Getting logs from block 13966166 to 14016165
Received 239 tx logs
Entries before address and topic filter: 239
Entries after filters: 202
Getting logs from block 14016166 to 14066165
Received 425 tx logs
Entries before address and topic filter: 425
Entries after filters: 317
Getting logs from block 14066166 to 14116165
Received 198 tx logs
Entries before address and topic filter: 198
Entries after filters: 155
Getting logs from block 14116166 to 14166165
Received 508 tx logs
Entries before address and topic filter: 508
Entries after filters: 377
Getting logs from block 14166166 to 14216165
Received 412 tx logs
Entries before address and topic filter: 412
Entries after filters: 300
Getting logs from block 14216166 to 14266165
Received 349 tx logs
Entries before address and topic filter: 349
Entries after filters: 270
Getting logs from block 14266166 t